# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [1]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1, notes="AAPL buy order")
        order2 = self.buy("GOOG", quantity2, notes="GOOG buy order")

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900314  89.141426  130.451989  89.429268  127.713306   
2022-12-27  128.097488  87.275429  129.456984  88.833741  126.806960   
2022-12-28  124.166786  85.816360  129.082622  87.861023  123.999315   
2022-12-29  127.683723  88.287827  128.540789  88.699738  125.831667   
2022-12-30  127.998978  88.069473  128.018685  88.168727  125.536140   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  86.967734  128.974282  86.967734  63814900  17815000  
2022-12-27  86.883373  129.427431  88.645153  69007800  15470900  
2022-12-28  85.727034  127.742834  86.848619  85438400  17879600  
2022-12-29  86.342418  126.087797  86.382121  75703700  18280700  
2022-12-30  86.382124  126.501579  86.714629  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

result.head()

Strategy started!
Bought 1239 AAPL and 946 GOOG
Strategy finished!


,BALANCE,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED_TOTAL,EQUITY
ITERATION,,,,
2018-01-02,16.905891,99983.094109,0,100000.000000
2018-01-03,16.905891,100795.057529,0,100811.963421
2018-01-04,16.905891,101211.215149,0,101228.121040
2018-01-05,16.905891,102526.010803,0,102542.916695
2018-01-08,16.905891,102558.526730,0,102575.432621


## <b> Backtest Statistics </b>

In [5]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 16.91
Final equity: 241921.36
---------------------------


In [6]:
bt.get_trade_statistics()

{'n_won': 1,
 'n_lost': 0,
 'n_total': 1,
 'win_rate': 1.0,
 'mean_profit': 141921.3609237671,
 'median_profit': 141921.3609237671,
 'mean_profit_win': 141921.3609237671,
 'mean_profit_loss': nan,
 'median_profit_win': 141921.3609237671,
 'median_profit_loss': nan,
 'std_profit': nan,
 'mad_profit': 0.0,
 'skewness': nan,
 'kurtosis': nan,
 'mean_median_gap': 0.0,
 'mean_profit_over_vol': nan,
 'median_profit_over_vol': nan}

In [7]:
bt.get_equity_statistics_dollar()

{'total_profit': 141921.3609237671,
 'mean_profit': 112.81507227644443,
 'median_profit': 256.20544052124023,
 'std_profit': 4002.7530759656993,
 'mad_profit': 2424.4220512466277,
 'skewness': -0.19350158755511118,
 'kurtosis': 3.056905923701165,
 'mean_median_gap': -143.3903682447958,
 'mean_profit_over_vol': 0.028184369641444044,
 'median_profit_over_vol': 0.06400730588644378,
 'max_drawdown': -122839.053855896}

In [8]:
bt.get_equity_statistics_return()

{'total_return': 1.419213609237671,
 'mean_return': 0.0008838017894162636,
 'median_return': 0.0016635736051274774,
 'std_return': 0.019027779637064507,
 'mad_return': 0.014074647172005976,
 'skewness': -0.15457326369243177,
 'kurtosis': 4.071149344215684,
 'mean_median_gap': -0.0007797718157112138,
 'mean_return_over_vol': 0.04644797271536046,
 'median_return_over_vol': 0.08742867727388311,
 'max_drawdown': -0.3432400529230194}

In [9]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,AGE,REASON_CLOSED,POSITIONS_FILLED,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED,POSITIONS_UNFILLED_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE
0,66e6da8b-b5cc-405e-b7a5-ee68c6847afc,OPEN,Buy and Hold Trade,141921.360924,2018-01-02,None,1823 days,None,"{'AAPL': {'quantity': 1239, 'value': 158590.73...",241904.455032,{},0,None,None,None


In [10]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,NOTES,TRADE_ID,ITERATION_REQUESTED,QUANTITY_REQUESTED,PRICE_REQUESTED,FEE_REQUESTED,GROSS_TOTAL_REQUESTED,TOTAL_REQUESTED,ITERATION_FILLED,QUANTITY_FILLED,PRICE_FILLED,FEE_FILLED,GROSS_TOTAL_FILLED,TOTAL_FILLED
0,341accc4-f9ac-4972-a976-6c8faa68882e,AAPL,BUY,FILLED,AAPL buy order,66e6da8b-b5cc-405e-b7a5-ee68c6847afc,2018-01-02,1239,40.341888,0,-49983.599762,-49983.599762,2018-01-02,1239,40.341888,0.0,-49983.599762,-49983.599762
1,00d40d1c-1851-4845-8b01-0ef0e5879a6b,GOOG,BUY,FILLED,GOOG buy order,66e6da8b-b5cc-405e-b7a5-ee68c6847afc,2018-01-02,946,52.853588,0,-49999.494347,-49999.494347,2018-01-02,946,52.853588,0.0,-49999.494347,-49999.494347


In [11]:
bt.strategy.positions_filled

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.73332214355)},
 'GOOG': {'quantity': 946, 'value': np.float64(83313.72171020508)}}

In [12]:
bt.strategy.positions_unfilled

{}